In [ ]:
!pip install stepfunctions

* Step Functions workflow 생성을 위한 role 생성 필요

In [2]:
import sagemaker
from sagemaker import get_execution_role
import stepfunctions
from stepfunctions.steps import *
from stepfunctions.workflow import Workflow

sm_role = get_execution_role()
wf_role = 'arn:aws:iam::233037139193:role/StepFunctionsWorkflowExecutionRole'
input_uri = 's3://sagemaker-us-east-1-233037139193/mbp3/dataset/raw_data.csv'
dataset_path = 's3://sagemaker-us-east-1-233037139193/mbp3/dataset'
dataset_uri = dataset_path + '/dataset.pkl.gz'
bucket = 'sagemaker-us-east-1-233037139193'

In [2]:
from sagemaker.tensorflow import TensorFlow

sf_estimator = TensorFlow(entry_point='../01_Experiments/mnist_simple_nn.py',
                          role=sm_role,
                          train_instance_count=1,
                          train_instance_type='ml.m5.xlarge',
                          framework_version='2.1.0',
                          py_version='py3')

## Simple pipeline

In [3]:
from stepfunctions.template.pipeline import TrainingPipeline

pipeline = TrainingPipeline(estimator=sf_estimator,
                            role=wf_role,
                            inputs=dataset_uri,
                            s3_bucket=bucket)

In [4]:
pipeline.render_graph()

In [5]:
pipeline.create()

'arn:aws:states:us-east-1:233037139193:stateMachine:training-pipeline-2020-07-01-23-01-50'

In [6]:
execution = pipeline.execute()

In [7]:
execution.render_progress()

# Making each step

## Execution input define

In [8]:
from stepfunctions.inputs import ExecutionInput

execution_input = ExecutionInput(schema={
    'InputPath': str,
    'OutputPath': str,
    'ExecutionName': str
})

## From training to deploying

In [10]:
common_name = execution_input["ExecutionName"]

training_step = sagemaker.TrainingStep(
    state_id="Training",
    estimator=sf_estimator,
    job_name=common_name,
    data=dataset_uri
)
model_step = sagemaker.ModelStep(
    state_id="CreateModel",
    model=training_step.get_expected_model(),
    model_name=common_name,
    instance_type=training_step.estimator.train_instance_type
)

endpoint_conf_step = sagemaker.EndpointConfigStep(
    state_id='EndpointConfigure',
    endpoint_config_name=common_name,
    model_name=common_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge'
)

endpoint_step = sagemaker.EndpointStep(
    state_id='Deploy',
    endpoint_name=common_name,
    endpoint_config_name=common_name
)

In [11]:
ml_steps_def = Chain([
    training_step,
    model_step,
    endpoint_conf_step,
    endpoint_step
])

## Preprocessing step using lambdas

In [12]:
create_processing_step = LambdaStep(
    state_id="StartPreprocessing",
    parameters={  
        "FunctionName": "arn:aws:lambda:us-east-1:233037139193:function:preprocessor9",
        "Payload": {
            "InputPath": execution_input['InputPath'],
            "OutputPath": execution_input['OutputPath'],
            "ExecutionName": execution_input['ExecutionName']
        }
    }
)

get_processing_status = LambdaStep(
    state_id="GetPreprocessingStatus",
    parameters={
        "FunctionName": "arn:aws:lambda:us-east-1:233037139193:function:checkstatus",
        "Payload": {
            "JobName": create_processing_step.output()['Payload']['JobName']
        }
    }
)

wait_state = Wait(
    state_id="Wait",
    seconds=60
)

## Choice step - loop to check the processing state

In [13]:
preprocessing_failure = Fail(
    state_id='Preprocessing Fail',
    error='Preprocessing has failed!',
    cause='See lambda error'
)
check_job_choice = Choice(
    state_id="IsPreProcessingComplete"
)
check_job_choice.add_choice(
    ChoiceRule.StringEquals(variable=get_processing_status.output()['Payload']['ProcessingJobStatus'],
                            value='InProgress'),
    next_step=get_processing_status
)

check_job_choice.add_choice(
    ChoiceRule.StringEquals(variable=get_processing_status.output()['Payload']['ProcessingJobStatus'],
                            value='Stopping'),
    next_step=get_processing_status
)

check_job_choice.add_choice(
    ChoiceRule.StringEquals(variable=get_processing_status.output()['Payload']['ProcessingJobStatus'],
                            value='Failed'),
    next_step=preprocessing_failure
)

check_job_choice.add_choice(
    ChoiceRule.StringEquals(variable=get_processing_status.output()['Payload']['ProcessingJobStatus'],
                            value='Stopped'),
    next_step=preprocessing_failure
)

check_job_choice.add_choice(
    ChoiceRule.StringEquals(variable=get_processing_status.output()['Payload']['ProcessingJobStatus'],
                            value='Completed'),
    next_step=ml_steps_def
)

In [14]:
ml_preprocess_def = Chain([
    create_processing_step,
    get_processing_status,
    wait_state,
    check_job_choice
])

In [16]:
full_wf = Workflow(
    name='my-workflow',
    definition=ml_preprocess_def,
    role=wf_role
)

create 시점에 workflow가 등록(배포)된다.

In [17]:
full_wf.create()

'arn:aws:states:us-east-1:233037139193:stateMachine:my-workflow'

In [24]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
full_execution = full_wf.execute(inputs={
    'InputPath': input_uri,
    'OutputPath': dataset_path,
    'ExecutionName': 'sfworkflow-{}'.format(timestamp)
})

In [27]:
full_execution.render_progress()

In [28]:
full_execution.list_events(html=True)
# html parsing sometimes fails then set html=False for raw output

# Data 업로드로 trigger

In [ ]:
import boto3
s3_client = boto3.client('s3')
s3_client.upload_file('../00_Basics/raw_data.csv', bucket, 'mbp3/dataset/raw_data.csv')

In [34]:
Workflow.list_workflows()

[{'stateMachineArn': 'arn:aws:states:us-east-1:233037139193:stateMachine:MyWorkflowFromScratch',
  'name': 'MyWorkflowFromScratch',
  'type': 'STANDARD',
  'creationDate': datetime.datetime(2020, 6, 28, 11, 24, 38, 716000, tzinfo=tzlocal())},
 {'stateMachineArn': 'arn:aws:states:us-east-1:233037139193:stateMachine:my-workflow',
  'name': 'my-workflow',
  'type': 'STANDARD',
  'creationDate': datetime.datetime(2020, 7, 1, 23, 13, 3, 194000, tzinfo=tzlocal())},
 {'stateMachineArn': 'arn:aws:states:us-east-1:233037139193:stateMachine:training-pipeline-2020-06-28-09-37-44',
  'name': 'training-pipeline-2020-06-28-09-37-44',
  'type': 'STANDARD',
  'creationDate': datetime.datetime(2020, 6, 28, 9, 45, 8, 219000, tzinfo=tzlocal())},
 {'stateMachineArn': 'arn:aws:states:us-east-1:233037139193:stateMachine:training-pipeline-2020-07-01-23-01-50',
  'name': 'training-pipeline-2020-07-01-23-01-50',
  'type': 'STANDARD',
  'creationDate': datetime.datetime(2020, 7, 1, 23, 1, 52, 831000, tzinfo=tzl

In [3]:
wf_ref = Workflow.attach(state_machine_arn='arn:aws:states:us-east-1:233037139193:stateMachine:my-workflow')

In [4]:
wf_ref.list_executions()

[Execution(execution_arn='arn:aws:states:us-east-1:233037139193:execution:my-workflow:80e69bb8-9c83-4fa4-8bd0-e965bc09e781', name='80e69bb8-9c83-4fa4-8bd0-e965bc09e781', status='ABORTED', start_date=datetime.datetime(2020, 7, 3, 0, 10, 20, 473000, tzinfo=tzlocal())),
 Execution(execution_arn='arn:aws:states:us-east-1:233037139193:execution:my-workflow:86d3373e-8d53-487b-9790-6469688a7530', name='86d3373e-8d53-487b-9790-6469688a7530', status='ABORTED', start_date=datetime.datetime(2020, 7, 3, 0, 10, 31, 337000, tzinfo=tzlocal())),
 Execution(execution_arn='arn:aws:states:us-east-1:233037139193:execution:my-workflow:757ca002-9431-4a41-87f0-2d317b826971', name='757ca002-9431-4a41-87f0-2d317b826971', status='FAILED', start_date=datetime.datetime(2020, 7, 3, 0, 2, 37, 374000, tzinfo=tzlocal())),
 Execution(execution_arn='arn:aws:states:us-east-1:233037139193:execution:my-workflow:a837c216-b074-433d-b63e-aa5fb605b4a7', name='a837c216-b074-433d-b63e-aa5fb605b4a7', status='SUCCEEDED', start_da

In [5]:
from stepfunctions.workflow import Execution
import datetime

In [6]:
exec_ref = Execution(workflow=wf_ref,
                     execution_arn='arn:aws:states:us-east-1:233037139193:execution:my-workflow:b887d6c3-0aa8-41f8-a026-52a5cf79a4be',
                     start_date=datetime.datetime(2020, 7, 2), status='RUNNING')

In [7]:
exec_ref.render_progress()